# fits数据的可视化及抠图

In [ ]:
# def sep_isbig(data):
#     import sep
#     # measure a spatially varying background on the image
#     bkg = sep.Background(data)
#     data_sub = data - bkg
#     objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
#     n=len(objects)
#     #作图检验
# #     image_show(data_sub,objects,m,s)
# #     print(n)
#     #只有一个星源，像素点数小于350,则保留下来
#     if n==1 and objects[0]['npix']<=350:
#         return 1
#     else:
#         return 0

In [ ]:
# #根据地址加载文件
# def loadfile(path):
#     import sys
#     import os

#     filenames=sorted(os.listdir(path)):
# #     for(dirpath,dirnames,files)in os.walk(path):
# #         filename=files
# #         for file in files:
# #             filepath.append(dirpath+"/"+file)
#     return filenames

In [1]:
#实现世界坐标转化为像素坐标
def world2pix(hdu,ra,dec):
    import astropy.wcs as wcs
    del hdu[0].header['RADECSYS']
    hdu[0].header['RADESYSa']='ICRS'
    w = wcs.WCS(hdu[0].header, hdu)
    hdu.close()
    x, y = w.all_world2pix(ra, dec, 1)

    return int(x),int(y)

In [50]:
#抠图
def acquire_image(path,ra,dec):
    from astropy.io import fits 
    import numpy as np
    hdu= fits.open(path)
    data=hdu[0].data
    data=data.byteswap().newbyteorder()
    print(data.shape)
    #转换坐标系
    x,y=world2pix(hdu,float(ra),float(dec))
    print("x:",x,"y:",y)
    #截图
    data1=data[y-10:y+10,x-10:x+10]
    return data1

In [3]:
import os
from astropy.io import fits 
import numpy as np
import pandas as pd
sample_path = "/home/qiao/文档/G盘/star/star_samples.csv"
image_path="/home/qiao/文档/G盘/star_images/"
path="/home/qiao/文档/G盘/star_bz2/"
#加载文件
bland=['u','g','r','i','z']
#抠图
file=pd.read_csv(sample_path)


In [6]:
for names in os.walk(path):
    names=names

In [14]:
err_indexs=[]
for temp in df:
    err_indexs.append(int(temp.split("_")[0]))

In [4]:
err_indexs=[14884]

In [102]:
data2=np.array(data[2])

In [103]:
print(data2.shape)

(20, 20)


In [101]:
data[2]=np.pad(data[2],((0,0),(0,11)),mode="reflect")

In [104]:
data[3]=data[4]

In [53]:
image=acquire_image("/home/qiao/文档/G盘/star_bz2/14884_i.fits.bz2",ra,dec)

(1489, 2048)
x: 2058 y: 1010


In [31]:
ra

0.0223367259284828

In [32]:
dec

34.535178615721804

In [28]:
image

array([], shape=(20, 0), dtype=float32)

In [94]:
star_indexs=[]#记录有图像的数据

# for i in range(22000,24007):
for i in err_indexs:
    count=0
    data=[]
    filepaths=[]
    ra=file.iloc[i]['ra']
    dec=file.iloc[i]['dec']
    for j in range(5):
        file_path=path+str(i)+'_'+bland[j]+'.fits.bz2'
        print(file_path)
        if os.path.exists(file_path):
            data.append(acquire_image(file_path,ra,dec))
            filepaths.append(file_path)
        else:
            count+=1
    if count==0:#五张图片都存在
        name=image_path+str(i)+".npy"
        star_indexs.append(i)
        print(len(data))
        if os.path.exists(name):
            os.remove(name)
            np.save(name,data)
        else:
            np.save(name,data)
        print(name)
        #保存data,并删除原有图像
#         for filepath in filepaths:
#             os.remove(filepath)
    elif count>0 and count<5:#有部分图像
#         err_indexs.append(i)
        pass
        

/home/qiao/文档/G盘/star_bz2/14884_u.fits.bz2
(1489, 2048)
x: 2012 y: 1071
/home/qiao/文档/G盘/star_bz2/14884_g.fits.bz2
(1489, 2048)
x: 2012 y: 1075
/home/qiao/文档/G盘/star_bz2/14884_r.fits.bz2
(1489, 2048)
x: 2049 y: 1018
/home/qiao/文档/G盘/star_bz2/14884_i.fits.bz2
(1489, 2048)
x: 2058 y: 1010
/home/qiao/文档/G盘/star_bz2/14884_z.fits.bz2
(1489, 2048)
x: 2011 y: 1072


In [33]:
#作图
def image_show(data_sub,objects,m,s):
    import matplotlib.pyplot as plt
    from matplotlib.patches import Ellipse
    import numpy as np
    n=len(objects)
    # plot background-subtracted image
    fig, ax = plt.subplots()
#     m, s = np.mean(data_sub), np.std(data_sub)
    im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
                   vmin=m-s, vmax=m+s, origin='lower')

    # plot an ellipse for each object
    for i in range(n):
        e = Ellipse(xy=(objects['x'][i], objects['y'][i]),#椭圆中心位置
                    width=8*objects['a'][i],#水平长度
                    height=8*objects['b'][i],#垂直长度
                    angle=objects['theta'][i] * 180. / np.pi)#旋转角度
        e.set_facecolor('blue')
        e.set_edgecolor('red')
        ax.add_artist(e)
    plt.show()

In [34]:
#观测下载图片
def sep_isbig(data):
    import sep
    # measure a spatially varying background on the image
    bkg = sep.Background(data)
    data_sub = data - bkg
    objects = sep.extract(data_sub, 1.5, err=bkg.globalrms)
    n=len(objects)
    m, s = np.mean(data), np.std(data)
    #作图检验
    image_show(data_sub,objects,m,s)

In [48]:
np.save("/home/qiao/文档/G盘/star_bz2/4.npy",data[3])
data0=np.load("/home/qiao/文档/G盘/star_bz2/4.npy")